# Transfer Functions example

Transfer functions are formated in various ways, this module attemps to make reading and writing between the various flavors easier.  Every format is read into a common container that can accommodate all metadata and statistical estimates.  This module makes no attempts to plot the data or analyze the data, that should be done using [MTpy v2.0](https://github.com/MTgeophysics/mtpy/tree/mpty_v2) (development to incorporate TF is in progress in version 2, version 1 mainly supports EDI files).  Here are some examples on how to use this module.

In [1]:
from mt_metadata.transfer_functions.core import TF

## Structure of TF
The TF object stores the data as an xarray.DataSet, metadata are stored in a `mt_metadata.transfer_functions.tf.Station` and `mt_metadata.transfer_functions.tf.Survey` metadata containers to create a standard container. 

In [18]:
tf_object = TF()

### Survey Metadata
The container for survey metadata is `survey_metadata`, this includes `citation` information, `project` information, and general information about the survey the transfer function was collected for.  

In [9]:
print(tf_object.survey_metadata.to_json(required=False))

{
    "survey": {
        "acquired_by.author": null,
        "acquired_by.comments": null,
        "citation_dataset.authors": null,
        "citation_dataset.doi": null,
        "citation_dataset.journal": null,
        "citation_dataset.pages": null,
        "citation_dataset.title": null,
        "citation_dataset.volume": null,
        "citation_dataset.year": "1980-01-01T00:00:00+00:00",
        "citation_journal.authors": null,
        "citation_journal.doi": null,
        "citation_journal.journal": null,
        "citation_journal.pages": null,
        "citation_journal.title": null,
        "citation_journal.volume": null,
        "citation_journal.year": "1980-01-01T00:00:00+00:00",
        "comments": null,
        "country": null,
        "datum": null,
        "fdsn.channel_code": null,
        "fdsn.id": null,
        "fdsn.network": null,
        "fdsn.new_epoch": null,
        "geographic_name": null,
        "id": null,
        "name": null,
        "northwest_corner.l

### Station Metadata
The container for station metadata is `station_metadata` this includes important `location` information, `orientation`, `provenance`, and `transfer_function` information. This also includes `run` information and `channel` information.

In [8]:
print(tf_object.station_metadata.to_json(required=False))

{
    "station": {
        "acquired_by.author": null,
        "acquired_by.comments": null,
        "channel_layout": null,
        "channels_recorded": [
            "hx",
            "ey",
            "hz",
            "ex",
            "hy"
        ],
        "comments": null,
        "data_type": null,
        "fdsn.channel_code": null,
        "fdsn.id": null,
        "fdsn.network": null,
        "fdsn.new_epoch": null,
        "geographic_name": null,
        "id": null,
        "location.datum": null,
        "location.declination.comments": null,
        "location.declination.epoch": null,
        "location.declination.model": "EMAG2",
        "location.declination.value": 0.0,
        "location.elevation": 0.0,
        "location.latitude": 0.0,
        "location.longitude": 0.0,
        "location.x": null,
        "location.x2": null,
        "location.y": null,
        "location.y2": null,
        "location.z": null,
        "location.z2": null,
        "orientation.angle_t

## Data Container

The data container is an xarray.DataSet and convenience methods are included to get/set `impedance`, `tipper`, and statistical estimates of errors.  This includes covariance estimates like those output by EMTF. 

The `dataset` is setup with `input` and `output` coordinates, for the sake of generality the default for `input` and `output` channels are ex, ey, hx, hy, hz.  Any input/output combo that does not have a value is set to nan.

### Input Channels

These are source channels, for natural source MT this will be hx and hy

### Output Channels

These are the response channels, for natural source MT this will be ex, ey, and hz.

In [10]:
tf_object.dataset

<xarray.Dataset>
Dimensions:                  (output: 5, input: 5, period: 1)
Coordinates:
  * output                   (output) <U2 'ex' 'ey' 'hx' 'hy' 'hz'
  * input                    (input) <U2 'ex' 'ey' 'hx' 'hy' 'hz'
  * period                   (period) int32 1
Data variables:
    transfer_function        (period, output, input) complex128 (nan+nanj) .....
    transfer_function_error  (period, output, input) float64 nan nan ... 0.0 nan
    inverse_signal_power     (period, output, input) complex128 (nan+nanj) .....
    residual_covariance      (period, output, input) complex128 0j 0j ... 0j
Attributes: (12/13)
    survey:          None
    project:         None
    id:              None
    name:            None
    latitude:        0.0
    longitude:       0.0
    ...              ...
    declination:     0.0
    datum:           None
    acquired_by:     None
    start:           1980-01-01T00:00:00+00:00
    end:             1980-01-01T00:00:00+00:00
    runs_processed:  ['None']

The `dataset` also has attributes that are the important information to describe a transfer function and commonly used to make inversion files.  

Use the convenience function `impedance`, `impedance_error`, `tipper`, `tipper_error` for accessing the common transfer function estimates. 

There are also functions for `has_` which informs you if that estimate exists.

In [32]:
print("\n".join([func for func in dir(tf_object) if callable(getattr(tf_object, func)) and not func.startswith("_")]))

copy
has_impedance
has_inverse_signal_power
has_residual_covariance
has_tipper
has_transfer_function
read_tf_file
write_tf_file


#### Set period range

**Important**: set the periods before seting any statistical estimate. 

In [19]:
n_periods = 6
tf_object.period = np.logspace(-3, 3, n_periods)

#### Set Impedance 

In [20]:
import numpy as np

tf_object.impedance = np.random.randn(n_periods, 2, 2) + np.random.randn(n_periods, 2, 2) * 1j

In [21]:
tf_object.has_impedance()

True

In [22]:
tf_object.impedance

<xarray.DataArray 'impedance' (period: 6, output: 2, input: 2)>
array([[[ 0.89710844+1.16531237j, -0.21295473+1.89940753j],
        [-0.25137739-0.57461505j, -0.93899833+0.14224552j]],

       [[-0.98459677-1.36220849j,  0.41389799-0.91763267j],
        [-1.60247932-1.06707602j,  0.83417378+0.73004218j]],

       [[ 1.99667661-0.24839079j, -1.54114158-0.22216879j],
        [-0.49514292+1.5643253j , -1.11987562-1.17843871j]],

       [[ 0.5833139 +1.37108964j, -0.46443252-0.92126433j],
        [ 1.18442493+0.29659547j,  1.01420307-0.49710336j]],

       [[ 0.44626621-0.34225008j,  1.20076213-0.27416824j],
        [ 0.8383837 -0.5543145j ,  0.86233904+1.73336183j]],

       [[-0.29365095-0.15416609j, -0.61505405-0.53097836j],
        [ 0.33858153+0.43572686j, -1.31127089-0.80220975j]]])
Coordinates:
  * output   (output) <U2 'ex' 'ey'
  * input    (input) <U2 'hx' 'hy'
  * period   (period) float64 0.001 0.01585 0.2512 3.981 63.1 1e+03

In [24]:
tf_object.dataset

<xarray.Dataset>
Dimensions:                  (output: 5, input: 5, period: 6)
Coordinates:
  * output                   (output) <U2 'ex' 'ey' 'hx' 'hy' 'hz'
  * input                    (input) <U2 'ex' 'ey' 'hx' 'hy' 'hz'
  * period                   (period) float64 0.001 0.01585 ... 63.1 1e+03
Data variables:
    transfer_function        (period, output, input) complex128 (nan+nanj) .....
    transfer_function_error  (period, output, input) float64 nan nan ... 0.0 nan
    inverse_signal_power     (period, output, input) complex128 (nan+nanj) .....
    residual_covariance      (period, output, input) complex128 0j 0j ... 0j
Attributes: (12/13)
    survey:          None
    project:         None
    id:              None
    name:            None
    latitude:        0.0
    longitude:       0.0
    ...              ...
    declination:     0.0
    datum:           None
    acquired_by:     None
    start:           1980-01-01T00:00:00+00:00
    end:             1980-01-01T00:00:00+00:00
    runs_processed:  ['None']

### Get impedance element
We can use xarray type indexing to get at elements

In [27]:
tf_object.impedance.loc[dict(input="hx", output="ey")][0]

<xarray.DataArray 'impedance' ()>
array(-0.25137739-0.57461505j)
Coordinates:
    output   <U2 'ey'
    input    <U2 'hx'
    period   float64 0.001